# 1. Use pandas to calculate a Transportation Distribution Charge for 3 Gas meters in the United Kingdom. 

In [2]:
%%timeit

from numpy import number, result_type
import pandas as pd
import time
import random
import numpy as np

file_location = r'C:\Users\dmcgregor001\Documents\gorilla_test\data.xlsx'

rates = pd.read_excel(file_location, sheet_name='Rates')
forecasted_consumption = pd.read_excel(file_location, sheet_name='Forecasted Consumption')
meter_list = pd.read_excel(file_location, sheet_name='Meter List')

# merge the meter list with the rates table based on the meters exit zone, we want each meter to have all of the potential rates at this stage
meter_list_with_rates = meter_list.merge(rates, left_on="Exit Zone", right_on="Exit Zone", how="inner")

# filter the list of meters based on the annual quantity in the meter list
meter_list_with_rates = meter_list_with_rates[(meter_list_with_rates['Annual Quantity (kWh)'] >= meter_list_with_rates['Annual Quantity (Min)']) & (meter_list_with_rates['Annual Quantity (kWh)'] < meter_list_with_rates['Annual Quantity (Max)'])]

# filter the list of meters based on the dates we have forecasts for to improve performance downstream
meter_list_with_rates = meter_list_with_rates[(meter_list_with_rates.Date >= forecasted_consumption.Date.min()) & (meter_list_with_rates.Date < forecasted_consumption.Date.max())]

# drop the AQ values now that we've already used them to find the rates
meter_list_with_rates = meter_list_with_rates.drop(['Annual Quantity (Min)', 'Annual Quantity (Max)'], axis=1)

#
# now we want to find the consumption charges for each period
#

# merge the forecast consumption schedule with the meter list to get potential rates for each forecast date
forecast_consumption_with_rates = forecasted_consumption.merge(meter_list_with_rates, on='Meter ID', how='left')

# filter the new forecast consumption with rates table to remove rate start dates that are after the current period date (rate start periods in the future don't apply today) 
forecast_consumption_with_rates = forecast_consumption_with_rates[(forecast_consumption_with_rates.Date_x >= forecast_consumption_with_rates.Date_y)]

# group the forecast consumption table by date and meter, exlcuding all but the highest rate start priod to ensure we are using the most current rate
forecast_consumption_with_rates = forecast_consumption_with_rates.drop_duplicates(subset=['Meter ID', 'Date_x'], keep='last')

# calculate the consumption cost per period
forecast_consumption_with_rates['daily_cost'] = forecast_consumption_with_rates['kWh'] * forecast_consumption_with_rates['Rate (p/kWh)']

#
# now we want to aggregate the consumption and costs for each of the meters and prepare an output frame
#

# group the forecast consumption table by meter and aggregate the forecast kWh consumption and daily cost per period
# this is technically not vectorised but it's still performant
output_dataframe = (forecast_consumption_with_rates.groupby([forecast_consumption_with_rates['Meter ID']]).agg(total_kwh=('kWh', sum), total_cost=('daily_cost', sum)))

# convert pence to pounds
output_dataframe['total_cost'] = output_dataframe['total_cost']/100 

# round to 2 decimal places
output_dataframe = output_dataframe.round(2)

# rename the headers to the appropriate labels
output_dataframe = output_dataframe.rename(columns={"total_kwh": "Total Estimated Consumption (kWh)", "total_cost": "Total Cost (£)"})

output_dataframe = output_dataframe.reset_index()

print(output_dataframe)

     Meter ID  Total Estimated Consumption (kWh)  Total Cost (£)
0    10588707                           122450.0           28.50
1    10626610                           303558.0           88.24
2  1000000603                            24674.0            8.27
     Meter ID  Total Estimated Consumption (kWh)  Total Cost (£)
0    10588707                           122450.0           28.50
1    10626610                           303558.0           88.24
2  1000000603                            24674.0            8.27
     Meter ID  Total Estimated Consumption (kWh)  Total Cost (£)
0    10588707                           122450.0           28.50
1    10626610                           303558.0           88.24
2  1000000603                            24674.0            8.27
     Meter ID  Total Estimated Consumption (kWh)  Total Cost (£)
0    10588707                           122450.0           28.50
1    10626610                           303558.0           88.24
2  1000000603            

# 2. Write a function that generates a list of random meters of any size. Examples of valid Exit Zones can be found in the rate table. You may randomly generate the Annual Quantity.

In [60]:
%%timeit

def generate_random_meters(number_of_meters, meter_id_length):
    # create a number of meters of a certain length based on the provided parameters
    meter_list = pd.DataFrame(random.sample(range(10**(meter_id_length-1),10**meter_id_length), number_of_meters), columns={"Meter ID"})
    
    # randomly generate some annual Wh consumption numbers and add them the dataframe 
    meter_list["Annual Quantity (kWh)"] = random.choices(range(0,100000), k=number_of_meters)
    
    # convert the Wh into kWh
    meter_list["Annual Quantity (kWh)"] = meter_list["Annual Quantity (kWh)"]/1000
    
    # randomly sample from the available exit zones and add them to each of the meters
    meter_list["Exit Zone"] = list(rates['Exit Zone'].sample(meter_list.count()[0], replace=True))
    
    return meter_list

print(generate_random_meters(number_of_meters=1000000, meter_id_length=10))

          Meter ID  Annual Quantity (kWh) Exit Zone
0       9404617390                 10.378       WA1
1       6173302201                 27.777       NW2
2       8613841636                 48.928       EA1
3       4322859584                 90.462        LT
4       6174282433                 83.478       SE1
...            ...                    ...       ...
999995  2584097476                 54.796       NE2
999996  8454291562                 90.273       NE2
999997  3911557295                 40.904       EA4
999998  2617278187                 68.565        LT
999999  6902948924                  7.014       EA3

[1000000 rows x 3 columns]
Wall time: 1.45 s


# 3. Write a function that generates mock consumption data given a list of meters and a start and end date.

In [61]:
%%timeit

def generate_random_consumption(list_of_meters, start_period, end_period):
    # create data frame of dates for the range provided in params
    random_forecast_consumption_date_range = pd.DataFrame(pd.date_range(start=start_period,end=end_period), columns=["Date"])

    # cross join meter list to date range to get date for each meter
    random_forecast_consumption = random_forecast_consumption_date_range.merge(list_of_meters, how='cross')

    # add random Wh to each date / meter combo
    random_forecast_consumption['kWh'] = random.choices(range(0,1000), k=random_forecast_consumption.shape[0])
    
    # convert Wh to kWh
    random_forecast_consumption['kWh'] = random_forecast_consumption['kWh']/1000
    
    
    return random_forecast_consumption

random_meter_list = generate_random_meters(number_of_meters=1000, meter_id_length=10)
print(generate_random_consumption(list_of_meters=random_meter_list["Meter ID"], start_period="2020-06-30", end_period="2021-06-30"))

             Date    Meter ID    kWh
0      2020-06-30  6993895507  0.826
1      2020-06-30  8000591730  0.443
2      2020-06-30  9584818432  0.888
3      2020-06-30  3758897175  0.669
4      2020-06-30  3051168636  0.850
...           ...         ...    ...
365995 2021-06-30  5025652425  0.650
365996 2021-06-30  7909602972  0.617
365997 2021-06-30  6933795349  0.634
365998 2021-06-30  2755904309  0.282
365999 2021-06-30  4208459231  0.006

[366000 rows x 3 columns]
Wall time: 177 ms


# 4. Write a function that takes as an input a meter list and a consumption table and returns the Transportation cost table. Benchmark this function using meter lists of different sizes. Try longer periods as well? How does the function scale? 

In [65]:
%%time

def generate_transport_cost_table(random_dataframe_of_meters, random_forecast_consumption_table): 
    # merge the meter list with the rates table based on the meters exit zone, we want each meter to have all of the potential rates at this stage
    meter_list_with_rates = random_dataframe_of_meters.merge(rates, left_on="Exit Zone", right_on="Exit Zone", how="inner")

    # filter the list of meters based on the annual quantity in the meter list
    meter_list_with_rates = meter_list_with_rates[(meter_list_with_rates['Annual Quantity (kWh)'] >= meter_list_with_rates['Annual Quantity (Min)']) & (meter_list_with_rates['Annual Quantity (kWh)'] < meter_list_with_rates['Annual Quantity (Max)'])]
    
    # filter the list of meters based on the dates we have forecasts for to improve performance downstream
    meter_list_with_rates = meter_list_with_rates[(meter_list_with_rates.Date >= random_forecast_consumption_table.Date.min()) & (meter_list_with_rates.Date < random_forecast_consumption_table.Date.max())]

    # drop the AQ values now that we've already used them to find the rates
    meter_list_with_rates = meter_list_with_rates.drop(['Annual Quantity (Min)', 'Annual Quantity (Max)'], axis=1)
    
    #
    # now we want to find the consumption charges for each period
    #

    # merge the forecast consumption schedule with the meter list to get potential rates for each forecast date
    forecast_consumption_with_rates = random_forecast_consumption_table.merge(meter_list_with_rates, on='Meter ID', how='left')

    # filter the new forecast consumption with rates table to remove rate start dates that are after the current period date (rate start periods in the future don't apply today) 
    forecast_consumption_with_rates = forecast_consumption_with_rates[(forecast_consumption_with_rates.Date_x >= forecast_consumption_with_rates.Date_y)]

    # group the forecast consumption table by date and meter, exlcuding all but the highest rate start priod to ensure we are using the most current rate
    forecast_consumption_with_rates = forecast_consumption_with_rates.drop_duplicates(subset=['Meter ID', 'Date_x'], keep='last')

    # calculate the consumption cost per period
    forecast_consumption_with_rates['daily_cost'] = forecast_consumption_with_rates['kWh'] * forecast_consumption_with_rates['Rate (p/kWh)']

    #
    # now we want to aggregate the consumption and costs for each of the meters and prepare an output frame
    #

    # group the forecast consumption table by meter and aggregate the forecast kWh consumption and daily cost per period
    # this is technically not vectorised but it's performant
    output_dataframe = (forecast_consumption_with_rates.groupby([forecast_consumption_with_rates['Meter ID']]).agg(total_kwh=('kWh', sum), total_cost=('daily_cost', sum)))

    # convert pence to pounds
    output_dataframe['total_cost'] = output_dataframe['total_cost']/100 

    # round to 2 decimal places
    output_dataframe = output_dataframe.round(2)
    
    # rename the headers to the appropriate labels
    output_dataframe = output_dataframe.rename(columns={"total_kwh": "Total Estimated Consumption (kWh)", "total_cost": "Total Cost (£)"})
    
    # reset the index of the frame for completeness
    output_dataframe = output_dataframe.reset_index()

    return(output_dataframe)

random_meter_list = generate_random_meters(number_of_meters=10, meter_id_length=10)
print(generate_transport_cost_table(random_dataframe_of_meters=random_meter_list, random_forecast_consumption_table=generate_random_consumption(list_of_meters=random_meter_list["Meter ID"], start_period="2020-06-30", end_period="2021-06-30")))


     Meter ID  Total Estimated Consumption (kWh)  Total Cost (£)
0  1206373851                             141.18            0.05
1  1987157631                             135.89            0.05
2  3200927859                             138.52            0.05
3  3333920762                             136.70            0.05
4  5209020695                             137.04            0.05
5  5794378256                             141.88            0.05
6  5953390703                             136.61            0.04
7  6714514155                             135.57            0.04
8  8215694087                             138.22            0.05
9  9327646991                             126.90            0.04
Wall time: 122 ms


## Benchmarking different meter list lengths

In [81]:
%%time

random_meter_list = generate_random_meters(number_of_meters=100, meter_id_length=10)
print(generate_transport_cost_table(random_dataframe_of_meters=random_meter_list, random_forecast_consumption_table=generate_random_consumption(list_of_meters=random_meter_list["Meter ID"], start_period="2020-06-30", end_period="2021-06-30")))

      Meter ID  Total Estimated Consumption (kWh)  Total Cost (£)
0   1269755565                             129.81            0.04
1   1356460821                             132.06            0.04
2   1458375896                             135.20            0.05
3   1637051923                             136.78            0.05
4   1675949318                             139.38            0.05
..         ...                                ...             ...
95  9580832092                             139.90            0.05
96  9606325197                             136.21            0.05
97  9806226123                             128.62            0.04
98  9906325916                             138.47            0.05
99  9984218221                             132.17            0.05

[100 rows x 3 columns]
Wall time: 76.3 ms


In [79]:
%%time

random_meter_list = generate_random_meters(number_of_meters=1000, meter_id_length=10)
print(generate_transport_cost_table(random_dataframe_of_meters=random_meter_list, random_forecast_consumption_table=generate_random_consumption(list_of_meters=random_meter_list["Meter ID"], start_period="2020-06-30", end_period="2021-06-30")))

       Meter ID  Total Estimated Consumption (kWh)  Total Cost (£)
0    1006229691                             135.51            0.05
1    1014223464                             133.90            0.04
2    1025178930                             136.66            0.05
3    1034792598                             135.81            0.05
4    1044618780                             132.63            0.04
..          ...                                ...             ...
995  9917055754                             142.48            0.05
996  9928767155                             131.29            0.04
997  9934237664                             141.24            0.05
998  9986159894                             128.78            0.04
999  9991044113                             130.67            0.04

[1000 rows x 3 columns]
Wall time: 534 ms


In [78]:
%%time

random_meter_list = generate_random_meters(number_of_meters=10000, meter_id_length=10)
print(generate_transport_cost_table(random_dataframe_of_meters=random_meter_list, random_forecast_consumption_table=generate_random_consumption(list_of_meters=random_meter_list["Meter ID"], start_period="2020-06-30", end_period="2021-06-30")))

        Meter ID  Total Estimated Consumption (kWh)  Total Cost (£)
0     1000217536                             133.84            0.04
1     1000332510                             135.92            0.04
2     1000368617                             127.66            0.04
3     1003179008                             140.34            0.05
4     1004501300                             137.44            0.04
...          ...                                ...             ...
9995  9996613500                             133.90            0.04
9996  9996781698                             135.91            0.04
9997  9997088736                             138.36            0.05
9998  9998560175                             135.90            0.05
9999  9999578145                             135.06            0.05

[10000 rows x 3 columns]
Wall time: 6.26 s


In [84]:
%%time

random_meter_list = generate_random_meters(number_of_meters=100000, meter_id_length=10)
print(generate_transport_cost_table(random_dataframe_of_meters=random_meter_list, random_forecast_consumption_table=generate_random_consumption(list_of_meters=random_meter_list["Meter ID"], start_period="2020-06-30", end_period="2021-06-30")))

         Meter ID  Total Estimated Consumption (kWh)  Total Cost (£)
0      1000127178                             135.60            0.05
1      1000177253                             132.16            0.04
2      1000390203                             130.69            0.04
3      1000428099                             140.82            0.04
4      1000562856                             131.28            0.04
...           ...                                ...             ...
99995  9999666040                             136.27            0.05
99996  9999766142                             128.26            0.04
99997  9999769271                             132.65            0.04
99998  9999882327                             137.72            0.05
99999  9999990278                             140.73            0.05

[100000 rows x 3 columns]
Wall time: 49.4 s


## Benchmarking different periods

In [82]:
%%time

random_meter_list = generate_random_meters(number_of_meters=1000, meter_id_length=10)
print(generate_transport_cost_table(random_dataframe_of_meters=random_meter_list, random_forecast_consumption_table=generate_random_consumption(list_of_meters=random_meter_list["Meter ID"], start_period="2020-06-30", end_period="2022-06-30")))

       Meter ID  Total Estimated Consumption (kWh)  Total Cost (£)
0    1003928416                             326.56            0.11
1    1019137200                             314.10            0.09
2    1027383538                             314.70            0.11
3    1028329166                             319.23            0.09
4    1031835514                             317.11            0.11
..          ...                                ...             ...
995  9871651977                             313.40            0.11
996  9984936690                             315.61            0.12
997  9991793789                             310.46            0.10
998  9992033187                             316.69            0.11
999  9994705051                             306.17            0.10

[1000 rows x 3 columns]
Wall time: 998 ms


In [83]:
%%time

random_meter_list = generate_random_meters(number_of_meters=1000, meter_id_length=10)
print(generate_transport_cost_table(random_dataframe_of_meters=random_meter_list, random_forecast_consumption_table=generate_random_consumption(list_of_meters=random_meter_list["Meter ID"], start_period="2020-06-30", end_period="2023-06-30")))

       Meter ID  Total Estimated Consumption (kWh)  Total Cost (£)
0    1016243672                             504.52            0.15
1    1049243771                             506.89            0.18
2    1077057517                             515.36            0.15
3    1077396302                             506.91            0.15
4    1082936402                             511.94            0.15
..          ...                                ...             ...
995  9960419081                             496.99            0.17
996  9960510125                             490.08            0.17
997  9979079715                             499.88            0.17
998  9982168737                             503.59            0.19
999  9992192072                             493.71            0.15

[1000 rows x 3 columns]
Wall time: 1.97 s


# 5. What are your observations after benchmarking? Are there any steps in the cost calculation that can be improved? How would you go about improving the performance of such a calculation?

The runtime of the transportation cost is more impacted by an increase in the number of meters being calculated vs an increase in the length of the time period. Cost calculation speed could potentially be improved by chunking and further leveraging multithreaded calculations. 